# Using tf-shell with Multiple Machines

`tf-shell` can be run on multiple machines using TensorFlow device placement.

A TensorFlow cluster is set up by each machine running something like the
following:

```python
cluster = tf.train.ClusterSpec('''{
  "alice": ["alice.com:2222"],
  "bob": ["bob.com:2223"],
}''')

server = tf.distribute.Server(
    cluster,
    job_name="/job:alice/replica:0/task:0/device:CPU:0",  # or bob
    task_index=0,
)

tf.config.experimental_connect_to_cluster(cluster)
```

In this notebook, we will emulate distributed execution on a single machine by
using the special job name `/job:localhost/replica:0/task:0/device:CPU:0` for
both alice and bob, and skip the server setup.

In [1]:
alice = "/job:localhost/replica:0/task:0/device:CPU:0"
bob = "/job:localhost/replica:0/task:0/device:CPU:0"

Since `tf-shell` works with sensitive cryptographic material, it is important to
tell TensorFlow to only place ops on devices which were explicitly assigned,
for security reasons.

In [2]:
import tensorflow as tf
tf.config.set_soft_device_placement(False)

2025-01-27 18:07:37.768791: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-27 18:07:37.777673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738001257.788038   61997 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738001257.790964   61997 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-27 18:07:37.800924: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

TensorFlow makes it easy to schedule operations on specific parties. In this
example, Alice will generate a secret key, encrypt the input x, and send it to
Bob. Bob will square the value, and return to alice, who will decrypt it.

In [ ]:
import tf_shell

@tf.function
def example_protocol(x):
  with tf.device(alice):
    shell_context = tf_shell.create_autocontext64(
        log2_cleartext_sz=6,
        scaling_factor=1,
        noise_offset_log2=0,
    )
    key = tf_shell.create_key64(shell_context)

    enc_x = tf_shell.to_encrypted(x, key, shell_context)

  with tf.device(bob):
    enc_x_squared = enc_x * enc_x
  
  with tf.device(alice):
    x_squared = tf_shell.to_tensorflow(enc_x_squared, key)
    return x_squared

# Turn on shell graph optimizers and deferred execution to use autocontext.
tf_shell.enable_optimization()
tf.config.run_functions_eagerly(False)

res = example_protocol(tf.constant([5.0]))
print(res[0])

2025-01-27 18:07:43.477964: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


ValueError: in user code:

    File "/tmp/ipykernel_61997/1727951566.py", line 14, in example_protocol  *
        key = tf_shell.create_key64(shell_context, param_cache)
    File "/workspaces/tf-shell/.venv/lib/python3.10/site-packages/tf_shell/python/shell_key.py", line 32, in create_key64  *
        raise ValueError(

    ValueError: A `cache_path` must be provided when `read_from_cache` is True.


In this example, we used the `cache_path` arguments when creating the context
and key. This prevents regenerating the parameters every time the code is run.
If we call the function again, these parameters will be loaded from the cache.

In [4]:
res = example_protocol(tf.constant([6.0]))
print(res[0])

INFO: Generating key
tf.Tensor(36.0, shape=(), dtype=float32)
